In [1]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error,mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pathlib


In [2]:
train=pd.read_csv("Z:\\smart_premium\\data\\raw\\train.csv")
test=pd.read_csv("Z:\\smart_premium\\data\\raw\\test.csv")
sample_submission=pd.read_csv("Z:\\smart_premium\\data\\raw\\sample_submission.csv")

In [3]:
train['Policy Start Date']=pd.to_datetime(train['Policy Start Date'])
train['Policy Start Year']=train['Policy Start Date'].dt.year
train['Policy Start Month']=train['Policy Start Date'].dt.month
train['Policy Start Day']=train['Policy Start Date'].dt.day
train['Day of week']=train['Policy Start Date'].dt.dayofweek
train=train.drop(columns=['Policy Start Date'])

In [4]:
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,Policy Start Year,Policy Start Month,Policy Start Day,Day of week
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,5.0,Poor,No,Weekly,House,2869.0,2023,12,23,5
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,2.0,Average,Yes,Monthly,House,1483.0,2023,6,12,0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,3.0,Good,Yes,Weekly,House,567.0,2023,9,30,5
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,Poor,Yes,Daily,Apartment,765.0,2024,6,12,2
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,4.0,Poor,Yes,Weekly,House,2022.0,2021,12,1,2


In [5]:
X=train.drop(columns=['id','Premium Amount','Customer Feedback'])
y=train['Premium Amount']

In [6]:
print(X.columns.tolist())

['Age', 'Gender', 'Annual Income', 'Marital Status', 'Number of Dependents', 'Education Level', 'Occupation', 'Health Score', 'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration', 'Smoking Status', 'Exercise Frequency', 'Property Type', 'Policy Start Year', 'Policy Start Month', 'Policy Start Day', 'Day of week']


In [7]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=21)

In [8]:
print(X_train)

          Age  Gender  Annual Income Marital Status  Number of Dependents  \
350978   19.0    Male         7096.0         Single                   0.0   
605645   37.0    Male         6072.0       Divorced                   3.0   
324121   20.0    Male         6897.0         Single                   0.0   
321404   56.0  Female        69929.0       Divorced                   0.0   
1130364  59.0    Male        17076.0       Divorced                   NaN   
...       ...     ...            ...            ...                   ...   
566392   45.0    Male        17287.0        Married                   3.0   
63856    60.0    Male         2862.0        Married                   0.0   
1130544  38.0    Male       102298.0         Single                   2.0   
664324   31.0  Female        16098.0         Single                   2.0   
202552   49.0    Male        25806.0         Single                   2.0   

        Education Level     Occupation  Health Score  Location    Policy Ty

In [9]:
num_cols=X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols=X.select_dtypes(include=['object','category']).columns.tolist()

num_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

col_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor=ColumnTransformer(transformers=[
    ('num',num_pipeline,num_cols),
    ('cat',col_pipeline,cat_cols)
])

In [10]:
models={
    'Linear Regression':LinearRegression(),
    'Decision Tree':DecisionTreeRegressor(random_state=30),
    'Random Forest':RandomForestRegressor(random_state=30,n_estimators=100,n_jobs=-1),
    'XGBoost':XGBRegressor(random_state=30,n_estimators=100,learning_rate=0.1,max_depth=6,n_jobs=-1,verbosity=0)
}

In [11]:
# =========================
# 3️⃣ MLflow Setup
# =========================
tracking_path = pathlib.Path(r"Z:\smart_premium\mlruns").absolute()
mlflow.set_tracking_uri(f"file:///{tracking_path.as_posix()}")
mlflow.set_experiment("Insurance_Premium_Prediction")

# =========================
# 4️⃣ Local save path
# =========================
local_model_dir = r"Z:\smart_premium\models"
os.makedirs(local_model_dir, exist_ok=True)

# =========================
# 5️⃣ Best model tracking
# =========================
best_model_name = None
best_model = None
best_rmse = float('inf')

# =========================
# 6️⃣ Training loop
# =========================
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        # Define pipeline
        pipe = Pipeline(steps=[
            ('preprocessor', preprocessor),  # your ColumnTransformer
            ('model', model)
        ])
        
        # Fit on training data
        pipe.fit(X_train, y_train)
        
        # Evaluate on validation set
        y_pred = pipe.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_val, y_pred)
        
        # Log metrics & params to MLflow
        mlflow.log_param('model_name', model_name)
        mlflow.log_metric('mae', mae)
        mlflow.log_metric('rmse', rmse)
        mlflow.log_metric('r2', r2)
        
        # Optional: log model to MLflow
        input_example = X_train.iloc[:5]
        mlflow.sklearn.log_model(pipe, name='model', input_example=input_example)
        
        print(f"{model_name} --> RMSE: {rmse:.2f}, R²: {r2:.3f}, MAE: {mae:.2f}")
        
        # Track best model
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_name = model_name
            best_model = pipe

# =========================
# 7️⃣ Retrain best model on full dataset
# =========================
print(f"\n🎯 Best model: {best_model_name} with RMSE {best_rmse:.2f}")

# Use all training + validation data for final training
X_full = pd.concat([X_train, X_val])
y_full = pd.concat([y_train, y_val])
best_model.fit(X_full, y_full)

# Save final best model
final_model_path = os.path.join(local_model_dir, "best_model.pkl")
joblib.dump(best_model, final_model_path)
print(f"✅ Final best model saved at: {final_model_path}")

# =========================
# 8️⃣ Prepare submission
# =========================
save_dir = r"Z:\smart_premium\data\processed"
os.makedirs(save_dir, exist_ok=True)

X_test = test.drop(['id'], axis=1)
test_preds = best_model.predict(X_test)

submission = pd.DataFrame({
    'id': test['id'],
    'Premium Amount': test_preds
})

submission_file = os.path.join(save_dir, "submission.csv")
submission.to_csv(submission_file, index=False)
print(f"✅ Submission file saved at: {submission_file}")


2025/10/01 07:35:28 INFO mlflow.tracking.fluent: Experiment with name 'Insurance_Premium_Prediction' does not exist. Creating a new experiment.
C:\Users\tamiz\AppData\Roaming\Python\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Linear Regression --> RMSE: 862.52, R²: 0.002, MAE: 665.97


C:\Users\tamiz\AppData\Roaming\Python\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Decision Tree --> RMSE: 1241.10, R²: -1.065, MAE: 905.15


C:\Users\tamiz\AppData\Roaming\Python\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Random Forest --> RMSE: 856.20, R²: 0.017, MAE: 661.15


C:\Users\tamiz\AppData\Roaming\Python\Python313\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


XGBoost --> RMSE: 847.21, R²: 0.038, MAE: 646.97

🎯 Best model: XGBoost with RMSE 847.21
✅ Final best model saved at: Z:\smart_premium\models\best_model.pkl
✅ Submission file saved at: Z:\smart_premium\data\processed\submission.csv


In [12]:
mlflow ui

SyntaxError: invalid syntax (3574192917.py, line 1)

In [ ]:
print(pipe.feature_names_in_)